In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.image as img
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv("train.csv")

In [3]:
target = train_df[list(train_df.keys().drop("image_id"))]
img_size = 400

In [4]:
image_path = "images/"
train_image = []
for name in train_df["image_id"]:
    image = Image.open(image_path+name+".jpg")
    image = image.resize((img_size, img_size))
    train_image.append(image)
x_train = np.ndarray(shape=(len(train_image), img_size, img_size, 3), dtype=np.float32)
for i in range(len(train_image)):
    x_train[i] = img_to_array(train_image[i])
x_train /= 255

In [5]:
del train_image

In [6]:
print(x_train.shape)

(1821, 400, 400, 3)


In [7]:
y_train = train_df.loc[:, "healthy":].values
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.15, random_state=69)

In [8]:
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((1547, 400, 400, 3), (274, 400, 400, 3), (1547, 4), (274, 4))

In [9]:
smote = SMOTE(random_state=69)
x_train, y_train = smote.fit_resample(x_train.reshape((-1, img_size*img_size*3)), y_train)
x_train = x_train.reshape((-1, img_size, img_size, 3))


In [10]:
x_train.shape, y_train.sum(axis=0)

((2128, 400, 400, 3), array([532, 532, 532, 532]))

In [11]:
lr = ReduceLROnPlateau(
    monitor="val_accuracy",
    factor=0.5,
    patience=10,
    min_lr=0.00001,
    verbose=1
)
es = EarlyStopping(
    monitor="val_loss",
    patience=20
)


In [12]:
from keras.models import Model, Sequential, load_model, Input
from keras.layers import (Conv2D, Dense, MaxPooling2D, LeakyReLU, Reshape,
                          Flatten, Dropout, BatchNormalization)
from keras.regularizers import l2

In [14]:
from keras.applications import ResNet50
reg=0.0005


model_d121 = ResNet50(include_top=False, weights="imagenet",
                           input_shape=(img_size, img_size, 3))
model_d121.trainable = False

# model = Sequential()
# model.add(model_i)
# filter 32
filter=32
model = Conv2D(filter, kernel_size=(3,3), data_format="channels_last" ,activation="relu", kernel_regularizer=l2(reg))(model_d121.output)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = Conv2D(filter, kernel_size=(3,3), activation="relu", kernel_regularizer=l2(reg))(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = MaxPooling2D(pool_size=(2,2), padding="SAME")(model)
model = Dropout(0.25)(model)

# # filter 64
filter=64
model = Conv2D(filter, kernel_size=(3,3), data_format="channels_last" ,activation="relu", kernel_regularizer=l2(reg))(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = Conv2D(filter, kernel_size=(3,3), activation="relu", kernel_regularizer=l2(reg))(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = MaxPooling2D(pool_size=(2,2), padding="SAME")(model)
model = Dropout(0.25)(model)

# # #filter 128
# # filter = 128
filter=128
model = Conv2D(filter, kernel_size=(1,1), data_format="channels_last" ,activation="relu", input_shape=(img_size,img_size,3), kernel_regularizer=l2(reg))(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = Conv2D(filter, kernel_size=(1,1), activation="relu", kernel_regularizer=l2(reg))(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = MaxPooling2D(pool_size=(2,2), padding="SAME")(model)
model = Dropout(0.25)(model)

# # filter = 256
# filter=256
# model = Conv2D(filter, kernel_size=(3,3), data_format="channels_last" ,activation="relu", input_shape=(img_size,img_size,3), kernel_regularizer=l2(reg))(model)
# model = BatchNormalization(axis=-1, center=True, scale=False)(model)
# model = Conv2D(filter, kernel_size=(3,3), activation="relu", kernel_regularizer=l2(reg))(model)
# model = BatchNormalization(axis=-1, center=True, scale=False)(model)
# model = MaxPooling2D(pool_size=(2,2), padding="SAME")(model)
# model = Dropout(0.25)(model)


# # filter = 512
# filter=512
# model = Conv2D(filter, kernel_size=(3,3), data_format="channels_last" ,activation="relu", input_shape=(img_size,img_size,3), kernel_regularizer=l2(reg))(model)
# model = BatchNormalization(axis=-1, center=True, scale=False)(model)
# model = Conv2D(filter, kernel_size=(3,3), activation="relu", kernel_regularizer=l2(reg))(model)
# model = BatchNormalization(axis=-1, center=True, scale=False)(model)
# model = MaxPooling2D(pool_size=(2,2), padding="SAME")(model)
# model = Dropout(0.25)(model)

model = Flatten()(model)
model = Dense(256, activation="relu")(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = Dropout(0.5)(model)
model = Dense(64, activation="relu")(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)


output = Dense(4, activation="softmax")(model)
model_d121 = Model(inputs=model_d121.input, outputs=output)
model_d121.summary()
model_d121.compile(optimizer="adam", 
             loss="categorical_crossentropy",
             metrics=["accuracy"])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 400, 400, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 406, 406, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 200, 200, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 200, 200, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [15]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=45,
    shear_range=0.25,
    zoom_range=0.25,
    width_shift_range=0.25,
    height_shift_range=0.25,
    rescale=1/255,
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest"
)

In [17]:
history = model_d121.fit_generator(datagen.flow(x_train, y_train, batch_size=8),
                             epochs=500,
                             steps_per_epoch=x_train.shape[0]//8,
                             verbose=1,
                             callbacks=[es, lr],
                             validation_data=(x_val, y_val)
                             )

Epoch 1/500
266/266 [==============================] - 168s 630ms/step - loss: 1.9418 - accuracy: 0.2702 - val_loss: 1.6475 - val_accuracy: 0.3285
Epoch 2/500
266/266 [==============================] - 170s 640ms/step - loss: 1.7361 - accuracy: 0.2758 - val_loss: 1.7870 - val_accuracy: 0.0438
Epoch 3/500
266/266 [==============================] - 174s 654ms/step - loss: 1.7043 - accuracy: 0.2697 - val_loss: 1.7045 - val_accuracy: 0.0438
Epoch 4/500
266/266 [==============================] - 174s 653ms/step - loss: 1.6719 - accuracy: 0.2721 - val_loss: 1.8255 - val_accuracy: 0.0657
Epoch 5/500
266/266 [==============================] - 174s 656ms/step - loss: 1.6651 - accuracy: 0.2871 - val_loss: 1.6171 - val_accuracy: 0.2007
Epoch 6/500
266/266 [==============================] - 174s 655ms/step - loss: 1.6426 - accuracy: 0.2787 - val_loss: 1.6788 - val_accuracy: 0.2007
Epoch 7/500
266/266 [==============================] - 174s 655ms/step - loss: 1.6361 - accuracy: 0.2707 - val_loss: 3

Epoch 56/500
266/266 [==============================] - 174s 654ms/step - loss: 1.1321 - accuracy: 0.5085 - val_loss: 0.9248 - val_accuracy: 0.6825
Epoch 57/500
266/266 [==============================] - 175s 656ms/step - loss: 1.1423 - accuracy: 0.5075 - val_loss: 0.7897 - val_accuracy: 0.7153
Epoch 58/500
266/266 [==============================] - 175s 658ms/step - loss: 1.1030 - accuracy: 0.5381 - val_loss: 0.7903 - val_accuracy: 0.6934
Epoch 59/500
266/266 [==============================] - 175s 657ms/step - loss: 1.0921 - accuracy: 0.5296 - val_loss: 0.8539 - val_accuracy: 0.6971
Epoch 60/500
266/266 [==============================] - 175s 657ms/step - loss: 1.0467 - accuracy: 0.5465 - val_loss: 0.9092 - val_accuracy: 0.7044
Epoch 61/500
266/266 [==============================] - 175s 658ms/step - loss: 1.0425 - accuracy: 0.5479 - val_loss: 0.6872 - val_accuracy: 0.7445
Epoch 62/500
266/266 [==============================] - 174s 656ms/step - loss: 1.0546 - accuracy: 0.5602 - val_

Epoch 111/500
266/266 [==============================] - 175s 657ms/step - loss: 0.6079 - accuracy: 0.7918 - val_loss: 0.3829 - val_accuracy: 0.9088
Epoch 112/500
266/266 [==============================] - 175s 657ms/step - loss: 0.6001 - accuracy: 0.8040 - val_loss: 0.4421 - val_accuracy: 0.9416
Epoch 113/500
266/266 [==============================] - 174s 656ms/step - loss: 0.6144 - accuracy: 0.7895 - val_loss: 0.4144 - val_accuracy: 0.9234
Epoch 114/500
266/266 [==============================] - 175s 657ms/step - loss: 0.5953 - accuracy: 0.8036 - val_loss: 0.3641 - val_accuracy: 0.9453
Epoch 115/500
266/266 [==============================] - 175s 657ms/step - loss: 0.6094 - accuracy: 0.7993 - val_loss: 0.3748 - val_accuracy: 0.9562
Epoch 116/500
266/266 [==============================] - 175s 657ms/step - loss: 0.5529 - accuracy: 0.8134 - val_loss: 0.3502 - val_accuracy: 0.9526
Epoch 117/500
266/266 [==============================] - 175s 658ms/step - loss: 0.5944 - accuracy: 0.7942

Epoch 164/500
266/266 [==============================] - 175s 657ms/step - loss: 0.4149 - accuracy: 0.8778 - val_loss: 0.2842 - val_accuracy: 0.9489

Epoch 00164: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 165/500
266/266 [==============================] - 175s 657ms/step - loss: 0.3968 - accuracy: 0.8820 - val_loss: 0.2807 - val_accuracy: 0.9562
Epoch 166/500
266/266 [==============================] - 175s 659ms/step - loss: 0.4004 - accuracy: 0.8825 - val_loss: 0.2769 - val_accuracy: 0.9599
Epoch 167/500
266/266 [==============================] - 175s 656ms/step - loss: 0.4240 - accuracy: 0.8670 - val_loss: 0.2839 - val_accuracy: 0.9489
Epoch 168/500
266/266 [==============================] - 175s 657ms/step - loss: 0.4284 - accuracy: 0.8633 - val_loss: 0.2864 - val_accuracy: 0.9562
Epoch 169/500
266/266 [==============================] - 175s 657ms/step - loss: 0.3963 - accuracy: 0.8858 - val_loss: 0.3107 - val_accuracy: 0.9526
Epoch 170/500
266/266 [==================

In [ ]:
del x_train
del train_df

In [ ]:
test_df = pd.read_csv("test.csv")
test_ids = test_df["image_id"]
test_image = []
for name in test_df["image_id"]:
    image = Image.open(image_path+name+".jpg")
    image = image.resize((img_size, img_size))
    test_image.append(image)
x_test = np.ndarray(shape=(len(test_image), img_size, img_size, 3), dtype=np.float32)
for i in range(len(test_image)):
    x_test[i] = img_to_array(test_image[i])
x_test /= 255
del test_image

In [ ]:
pred = model_d121.predict(x_test)
res = pd.DataFrame()
res["image_id"]=test_ids
res["healthy"] = pred[:, 0]
res["multiple_diseases"] = pred[:, 1]
res["rust"] = pred[:, 2]
res["scab"] = pred[:, 3]
res.to_csv("submission_simple_resnet50_400.csv", index=False)

In [ ]:
res.head(10)

In [ ]:
import matplotlib.pyplot as plt
h = history.history
offset = 5
epochs = range(offset, len(h["loss"]))
plt.figure(1, figsize=(20,20))
plt.subplot(121)
plt.xlabel("epochs")
plt.ylabel("loss")
plt.plot(epochs, h["loss"][offset:], label="train")
plt.plot(epochs, h["val_loss"][offset:], label="val")
plt.legend()
plt.subplot(122)
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.plot(h[f'accuracy'], label='train')
plt.plot(h[f'val_accuracy'], label='val')
plt.legend()

In [ ]:
import pandas as pd